In [2]:
import L0_card
import numpy as np

In [10]:
data = np.load('/home/gridsan/xmeng/Sparse_NN_shared/error_data.npz')

In [95]:
X,y,beta,r,k,alpha,lambda2,lambda1,beta_tilde1,beta_tilde2 = data['X'],data['y'],data['beta'],data['r'],data['k'],data['alpha'],data['lambda2'],data['lambda1'],data['beta_tilde1'],data['beta_tilde2']

In [96]:
del hard_thresholding_ls

In [97]:
def hard_thresholding_ls(y,X,beta,r,k,alpha,lambda1,lambda2,beta_tilde1,beta_tilde2,L,M=np.inf,sea_max_itr=5):
    
    _, p = X.shape
    support = np.where(beta!=0)[0]
    support_inv = np.where(beta==0)[0]
    XTr = X.T@r
    beta_sub2 = beta_tilde2 - beta
    grad = -XTr + alpha - 2*lambda2*beta_sub2
    grad_supp = np.zeros(p)
    grad_supp[support] = grad[support]
    
    max_suppinv = np.max(np.abs(grad[support_inv]))
    same_sign = np.sign(beta[support])==np.sign(grad[support])
    L_change = np.min(np.where( same_sign + (np.abs(grad[support])<max_suppinv),
         np.abs(beta[support])/(np.abs(grad[support])*(2*same_sign-1)+max_suppinv),np.inf))
   
    
    if p > 5*k:
        Xgrad = X[:,support]@grad[support]
    else:
        Xgrad = X@grad_supp
    
 
    opt_step = (grad_supp@alpha - r@Xgrad - 2*lambda2*beta_sub2@grad_supp)/(Xgrad@Xgrad+2*lambda2*grad_supp@grad_supp)
    argsort1 = np.argsort(-np.abs(beta))
    argsort2 = np.argsort(-np.abs(beta - (L_change/(1+1e-8))*grad))
    print("Difference between set is ",set(argsort1[:k])-set(argsort2[:k]))
    print("opt step is",opt_step,"Lchange is",L_change)
    if opt_step < L_change:
        beta_new = beta - opt_step*grad
        argsort = np.argsort(-np.abs(beta_new))
        beta_new[argsort[k:]] = 0
        if p > 5*k:
            r = y - X[:,argsort[:k]]@beta_new[argsort[:k]]
        else:
            r = y - X@beta_new
    else:
        f_best = L0_card.evaluate_obj(beta,r,alpha,lambda1,lambda2,beta_tilde1,beta_tilde2)   
        sea_itr = 0
        L_step = L_change / (1+1e-6)
        while sea_itr < sea_max_itr:
            
            beta_tmp = beta - L_step*grad
            argsort = np.argsort(-np.abs(beta_tmp))
            beta_tmp[argsort[k:]] = 0
            if p > 5*k:
                r_tmp = y - X[:,argsort[:k]]@beta_tmp[argsort[:k]]
            else:
                r_tmp = y - X@beta_tmp
                
            f_new = L0_card.evaluate_obj(beta_tmp,r_tmp,alpha,lambda1,lambda2,beta_tilde1,beta_tilde2)
            print("f_new is",f_new,"f_best is",f_best)
            if f_new < f_best:
                f_best = f_new
                beta_new = np.copy(beta_tmp)
                r = np.copy(r_tmp)
            else:
                break
         
            L_step *= 2
            sea_itr += 1
    
    return beta_new, r

In [88]:
hard_thresholding_ls(y,X,beta,r,k,alpha,lambda1,lambda2,beta_tilde1,beta_tilde2,L=1e10,M=np.inf,sea_max_itr=5)

Difference between set is  set()
opt step is 4.442930952228918e-05 Lchange is 1.3871376285336638e-05
f_new is 3655.8011631506024 f_best is 4083.3042383126726
f_new is 197736318.77630213 f_best is 3655.8011631506024


(array([-0.55240339, -0.27406103,  0.39760795, ...,  0.22469325,
        -0.30517189, -0.3861283 ]),
 array([-3.72917456e-02, -2.45345309e-01, -1.07449528e-02,  1.93719188e+00,
         5.03987230e-04,  1.25922223e-04, -1.44777782e-01,  1.79405159e+00,
         6.40068726e-01, -3.72046245e+00, -1.89499468e-01, -5.55042511e+00,
        -1.72413827e-02,  2.42404110e-01,  3.16416084e+00, -2.32824420e-03,
         6.39424042e-01,  4.54432401e-03, -3.13880911e-01,  9.65531444e-01,
        -3.81754238e-03, -1.35494986e-01, -1.11579164e-03,  1.42935537e-02,
         1.59876371e-01, -8.56058512e-03, -8.48614254e-02,  7.84798708e-01,
         3.82943378e+00,  1.27870249e-01, -9.32098118e-04, -4.39323972e+00,
        -4.50171550e-02,  2.13357553e-02, -1.15733577e-01, -5.43374039e-02,
        -4.39246841e-03, -1.38639532e-01, -1.15736046e-02, -1.05644162e+00,
         5.68882017e-01,  4.31323261e-01, -1.69818256e-01,  7.98544576e-03,
        -8.39042916e-02, -1.28188544e+00,  1.86648280e-02, -7.8

In [98]:
opt_step = 4.442930952228918e-05
L_change = 1.3871376285336638e-05

In [99]:
_, p = X.shape
support = np.where(beta!=0)[0]
support_inv = np.where(beta==0)[0]
XTr = X.T@r
beta_sub2 = beta_tilde2 - beta
grad = -XTr + alpha - 2*lambda2*beta_sub2
grad_supp = np.zeros(p)
grad_supp[support] = grad[support]

In [49]:
beta1 = beta - opt_step*grad_supp
argsort = np.argsort(-np.abs(beta1))
beta1[argsort[k:]] = 0
r = y - X@beta1
L0_card.evaluate_obj(beta1,r,alpha,lambda1,lambda2,beta_tilde1,beta_tilde2)   

3272.0223382452477

In [51]:
beta1 = beta - (opt_step-1e-6)*grad_supp
argsort = np.argsort(-np.abs(beta1))
beta1[argsort[k:]] = 0
r = y - X@beta1
L0_card.evaluate_obj(beta1,r,alpha,lambda1,lambda2,beta_tilde1,beta_tilde2) 

3272.433329573976

In [101]:
beta1 = beta - (L_change)*grad
beta2 = beta - (L_change-1e-8)*grad
argsort1 = np.argsort(-np.abs(beta1))
argsort2 = np.argsort(-np.abs(beta2))
beta1[argsort1[k:]] = 0
beta2[argsort2[k:]] = 0
r1 = y - X@beta1
r2 = y - X@beta2
loss1 = L0_card.evaluate_obj(beta1,r1,alpha,lambda1,lambda2,beta_tilde1,beta_tilde2) 
loss2 = L0_card.evaluate_obj(beta2,r2,alpha,lambda1,lambda2,beta_tilde1,beta_tilde2)
loss1,loss2

(6836.811204523193, 3656.0520367391955)

In [103]:
r1@r1

9021.199532726936

In [104]:
r2@r2

2298.575666796023

In [102]:
set(argsort1[:k])-set(argsort2[:k])

{16500}

In [89]:
np.max(np.abs(beta))

2.3308865600844872

In [54]:
argsort1 = np.argsort(-np.abs(beta))
argsort2 = np.argsort(-np.abs(beta - (L_change/(1+1e-8))*grad))
print("Difference between set is ",set(argsort1[:k])-set(argsort2[:k]))

Difference between set is  set()


In [ ]:
beta_tmp = beta - L_change*grad
argsort = np.argsort(-np.abs(beta_tmp))
beta_tmp[argsort[k:]] = 0
r_tmp = y - X@beta_tmp
L0_card.evaluate_obj(beta_tmp,r_tmp,alpha,lambda1,lambda2,beta_tilde1,beta_tilde2)

In [26]:
L0_card.evaluate_obj(beta,r,alpha,lambda1,lambda2,beta_tilde1,beta_tilde2)   

array(50.)

In [ ]:
L0_card.hard_thresholding_ls(y,X,beta,r,k,alpha,lambda1,lambda2,beta_tilde1,beta_tilde2,L,M=np.inf,sea_max_itr=5)